In [1]:
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.visualization import quantity_support
import matplotlib.pyplot as plt
from gammapy.irf import (
    IRF,
    Background3D,
    EffectiveAreaTable2D,
    EnergyDependentMultiGaussPSF,
    EnergyDispersion2D,
    Background2D,
    RadMax2D,
)
from gammapy.irf.io import COMMON_IRF_HEADERS, IRF_DL3_HDU_SPECIFICATION
from gammapy.makers.utils import (
    make_edisp_kernel_map,
    make_map_exposure_true_energy,
    make_psf_map,
)
from gammapy.maps import MapAxis, WcsGeom
from astropy.io import fits
from ctlearn_manager import CTLearnTriModelManager
from ctlearn_manager.io import load_model_from_index

In [2]:
MODEL_INDEX_FILE = "/home/blacave/CTLearn/Software/CTLearn-Manager/ctearn_models_index.h5"
energy_model = load_model_from_index("energy_stereo_20deg", MODEL_INDEX_FILE)
direction_model = load_model_from_index("direction_stereo_20deg", MODEL_INDEX_FILE)
type_model = load_model_from_index("type_stereo_20deg", MODEL_INDEX_FILE)
Stereo_Tri_Model = CTLearnTriModelManager(direction_model=direction_model, energy_model=energy_model, type_model=type_model)

🧠 Model name: energy_stereo_20deg
🧠 Model name: direction_stereo_20deg
🧠 Model name: type_stereo_20deg


In [3]:
config = "/home/blacave/CTLearn/Software/CTLearn-Manager/src/ctlearn_manager/resources/public-conf.yml"

In [4]:
Stereo_Tri_Model.produce_irfs(20, 0)

💾 Model direction_stereo_20deg IRF data update:
💾 Model energy_stereo_20deg IRF data update:
💾 Model type_stereo_20deg IRF data update:


2025-01-06 09:01:43,061 INFO [ctapipe.ctapipe-optimize-event-selection] (tool.initialize): Loading config from '[PosixPath('/home/blacave/CTLearn/Software/CTLearn-Manager/src/ctlearn_manager/resources/public-conf.yml')]'
2025-01-06 09:01:43,067 DEBUG [ctapipe.core.provenance] (provenance.add_input_file): added input entity '/home/blacave/CTLearn/Software/CTLearn-Manager/src/ctlearn_manager/resources/public-conf.yml' to activity: 'ctapipe-optimize-event-selection'
2025-01-06 09:01:43,069 INFO [ctapipe.ctapipe-optimize-event-selection] (tool.initialize): ctapipe version 0.23.1.dev174+g8f481f3f
2025-01-06 09:01:43,071 DEBUG [ctapipe.ctapipe-optimize-event-selection] (tool.run): CONFIG: {'EventSelectionOptimizer': {'chunk_size': 100000, 'config_files': [PosixPath('/home/blacave/CTLearn/Software/CTLearn-Manager/src/ctlearn_manager/resources/public-conf.yml')], 'electron_file': None, 'electron_target_spectrum': <Spectra.IRFDOC_ELECTRON_SPECTRUM: 2>, 'gamma_file': PosixPath('/home/blacave/CTL

In [3]:
Stereo_Tri_Model.plot_irfs(20, 0)

KeyError: "Extension 'EFFECTIVE AREA' not found."

In [8]:
irf_filename = "/home/blacave/CTLearn/Data/CTLearn_Models_SST1M/IRFs/20deg70_stereo_IRFs.fits"

rad_max = RadMax2D.read(irf_filename, hdu="RAD_MAX")
aeff = EffectiveAreaTable2D.read(irf_filename, hdu="EFFECTIVE AREA")
bkg = Background2D.read(irf_filename, hdu="BACKGROUND")
edisp = EnergyDispersion2D.read(irf_filename, hdu="ENERGY DISPERSION")
edisp.peek()
aeff.peek()
bkg.peek()
aeff

NameError: name 'edisp' is not defined

In [12]:
hudl = fits.open(irf_filename)
hudl.info()
hudl['SENSITIVITY'].data

Filename: /home/blacave/CTLearn/Data/CTLearn_Models_SST1M/IRFs/20deg70_stereo_IRFs.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  ENERGY BIAS RESOLUTION    1 BinTableHDU     34   1R x 7C   [40D, 40D, D, D, 40D, 40D, 40D]   
  2  ANGULAR RESOLUTION    1 BinTableHDU     33   1R x 6C   [20D, 20D, D, D, 20D, 20D]   
  3  SENSITIVITY    1 BinTableHDU     52   1R x 12C   [20D, 20D, D, D, 20D, 20D, 20D, 20D, 20D, 20D, 20D, 20D]   


FITS_rec([([5.00000000e-02, 7.92446596e-02, 1.25594322e-01, 1.99053585e-01, 3.15478672e-01, 5.00000000e-01, 7.92446596e-01, 1.25594322e+00, 1.99053585e+00, 3.15478672e+00, 5.00000000e+00, 7.92446596e+00, 1.25594322e+01, 1.99053585e+01, 3.15478672e+01, 5.00000000e+01, 7.92446596e+01, 1.25594322e+02, 1.99053585e+02, 3.15478672e+02], [7.92446596e-02, 1.25594322e-01, 1.99053585e-01, 3.15478672e-01, 5.00000000e-01, 7.92446596e-01, 1.25594322e+00, 1.99053585e+00, 3.15478672e+00, 5.00000000e+00, 7.92446596e+00, 1.25594322e+01, 1.99053585e+01, 3.15478672e+01, 5.00000000e+01, 7.92446596e+01, 1.25594322e+02, 1.99053585e+02, 3.15478672e+02, 5.00000000e+02], [0.], [1.], [[          nan,           nan,           nan,           nan,           nan,           nan,  307.30025587,  406.7511208 ,  313.32306671,  347.85884917,  397.70904212,  402.49831695,  487.47667283,  615.72662164,  784.4834712 , 1030.985837  , 1366.32963908, 1689.55985999,           nan,           nan]], [[        nan,         nan,  